<a href="https://colab.research.google.com/github/shadowbd/Airline_delay_/blob/main/Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

#import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge, LinearRegression, ElasticNet, Lasso
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

# displaying all the columns and row throughout the notebook
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
df = pd.read_csv('modelling_major_option.csv')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
Categorial_col = ['brand', 'transmission', 'color', 'drivetrain', 'fuel type', 'engine-type', 'location', 'car-type']

In [ ]:
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
encoder_fit = OH_encoder.fit_transform(df[Categorial_col])
column_name = OH_encoder.get_feature_names(Categorial_col)
OH_cols = pd.DataFrame(encoder_fit, columns=column_name)
# One-hot encoding removed index; put it back
OH_cols.index = df.index
# Remove categorical columns (will replace with one-hot encoding)
num_col = df.drop(Categorial_col, axis=1)

In [ ]:
# y = num_col['price']

In [ ]:
# num_col.drop('price', axis=1, inplace = True)
# num_col.head()

In [ ]:
# sc=StandardScaler()

In [ ]:
# df_s = sc.fit_transform(num_col)

In [ ]:
# df_after_scaled = pd.DataFrame(df_s)

In [ ]:
num_col.columns

In [ ]:
# df_after_scaled.columns = ['mileage', 'age', 'engine-hp', 'engine-l', 'no_of_features', 'price']
# df_after_scaled.head()

In [ ]:
OH_df = pd.concat([num_col, OH_cols], axis=1)

In [ ]:
OH_df.head()

In [ ]:
y=OH_df['price']
X= OH_df.drop('price', axis = 1)

In [ ]:
X

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_train.shape

In [ ]:
lm = LinearRegression()
lm_scores = -1*cross_val_score(lm, X_train, y_train, scoring = 'neg_mean_absolute_error', cv = 5)
print("MAE score: {} and standard deviation {}".format(round(np.mean(lm_scores)), 
                                                        round(np.std(lm_scores))))

In [ ]:
rr = Ridge(alpha=0.01)
rr_scores = -1*cross_val_score(rr, X_train, y_train, scoring = 'neg_mean_absolute_error', cv = 5)
print("MAE score: {} and standard deviation {}".format(round(np.mean(rr_scores)), 
                                                        round(np.std(rr_scores))))

In [ ]:
xg = XGBRegressor()
xg_scores = -1*cross_val_score(xg, X_train, y_train, scoring = 'neg_mean_absolute_error', cv = 5)
print("MAE score: {} and standard deviation {}".format(round(np.mean(xg_scores)), 
                                                        round(np.std(xg_scores))))

In [ ]:
import xgboost as xgb
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [ ]:
xg_reg.fit( X_train, y_train)

In [ ]:
preds = xg_reg.predict(X_valid)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
mse = mean_squared_error(y_valid, preds)
print('MSE: %f' %(mse))

In [ ]:
rmse = np.sqrt(mean_squared_error(y_valid, preds))
print('RMSE: %f' %(rmse))

In [ ]:
from sklearn.metrics import r2_score
r2_score_xgb = r2_score(y_valid, preds)
print(r2_score_xgb)

GRID SEARCH

In [ ]:
param_grid_XG = {
    'objective': ['reg:squarederror'],
    'colsample_by_tree': [0.4, 0.5, 0.6],
    'learning_rate': [0.001, 0.01, 0.1, 0.5, 1],
    'max_depth': [3, 5, 7, 10, 15],
    'alpha': [1, 2, 3, 5, 10],
    'n_estimators': [5, 10, 30, 50]}

In [ ]:
xg_reg_gs = xgb.XGBRegressor()

In [ ]:
grid_XG = GridSearchCV(estimator=xg_reg_gs, param_grid=param_grid_XG, scoring='r2', cv=5, verbose=1, n_jobs=-1)

In [ ]:
grid_XG.fit(X_train, y_train)

In [ ]:
y_pred_XG = grid_XG.predict( X_valid)

In [ ]:
mae_xg = mean_absolute_error(y_valid, y_pred_XG)
mse_xg = np.sqrt(mean_squared_error(y_valid, y_pred_XG))
#r2_xg =metrics.r2_score(y_valid, y_pred_XG)

In [ ]:
print('Mean Absolute Error: ', mae_xg)
print('Mean Square Error: ', mse_xg)